In [2]:
import os
import argparse

from time import time

import psycopg2 
import pandas as pd 
from sqlalchemy import create_engine 

In [5]:
#user = params.user
user = 'postgres'
#password = params.password
password = 'postgres'
#host = params.host 
host = 'localhost'
#port = params.port 
port = 5433
#db = params.db
db = 'ny_taxi'
#table_name = params.table_name
table_name = 'dim_zones'
#url = params.url

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

#conn = engine.connect() 

In [75]:
df = pd.read_parquet("../data/green_tripdata_2025-11.parquet", engine = 'fastparquet')
#df['lpep_pickup_date'] = pd.to_datetime(df['lpep_pickup_datetime']).dt.date
#df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime']).dt.date
datetime_cols = df.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns
for col in datetime_cols:
    df[col] = df[col].dt.normalize()
df.columns.astype

<bound method Index.astype of Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'cbd_congestion_fee'],
      dtype='str')>

### Question 3

In [35]:
dq3 = df.copy()
dq3 = dq3[(dq3['lpep_pickup_datetime']>'2025-11-01') &
    (dq3['lpep_pickup_datetime']<='2025-12-01') &
    (dq3['trip_distance']<=1)    
]

dq3.describe()
dq3.VendorID.count()
#dq3.head()

np.int64(8007)

### Question 4

In [31]:
dq4 = df.copy()
dq4 = dq4[dq4['trip_distance']<100]
max_trip_distance = dq4['trip_distance'].idxmax()

pickup_datetime = dq4.loc[max_trip_distance]

print("\n Pick up date with max trip distance: ")
print(pickup_datetime.lpep_pickup_datetime)


 Pick up date with max trip distance: 
2025-11-14 15:36:27


### Question 5

In [38]:
df_zones = pd.read_csv("../data/ntaxi_zone_lookup.csv")
df_zones.columns

Index(['LocationID', 'Borough', 'Zone', 'service_zone'], dtype='str')

In [91]:
dq5_tmp = df.copy()


dq5 = dq5_tmp.merge(df_zones,left_on='PULocationID', right_on="LocationID", how='left' )

dq5.tail()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee,LocationID,Borough,Zone,service_zone
46907,2,2025-11-30,2025-11-30,None,NaN,59,51,NaN,8.50,33.22,...,1.0,34.72,NaN,NaN,NaN,0.00,59,Bronx,Crotona Park,Boro Zone
46908,2,2025-11-30,2025-11-30,None,NaN,74,151,NaN,1.73,13.86,...,1.0,16.13,NaN,NaN,NaN,0.00,74,Manhattan,East Harlem North,Boro Zone
46909,2,2025-11-30,2025-11-30,None,NaN,33,163,NaN,7.52,38.42,...,1.0,44.42,NaN,NaN,NaN,0.75,33,Brooklyn,Brooklyn Heights,Boro Zone
46910,2,2025-11-30,2025-11-30,None,NaN,16,95,NaN,5.61,24.67,...,1.0,26.17,NaN,NaN,NaN,0.00,16,Queens,Bayside,Boro Zone
46911,2,2025-11-30,2025-11-30,None,NaN,65,87,NaN,2.91,14.46,...,1.0,23.35,NaN,NaN,NaN,0.75,65,Brooklyn,Downtown Brooklyn/MetroTech,Boro Zone


In [92]:
date_q5 = pd.to_datetime(['2025-11-18'])
#dq5 = dq5[dq5['lpep_pickup_datetime']>='2025-11-18']
#dq5 = dq5[dq5['lpep_pickup_datetime']=='2025-11-18']
dq5 = dq5[dq5['lpep_pickup_datetime'].dt.normalize().isin(date_q5)]

dq5.head()


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee,LocationID,Borough,Zone,service_zone
22419,2,2025-11-18,2025-11-18,N,1.0,74,75,1.0,1.40,12.1,...,1.0,17.00,1.0,1.0,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone
22420,2,2025-11-18,2025-11-18,N,1.0,75,236,1.0,1.33,10.0,...,1.0,17.10,1.0,1.0,2.75,0.0,75,Manhattan,East Harlem South,Boro Zone
22421,2,2025-11-18,2025-11-18,N,1.0,74,236,1.0,1.97,12.1,...,1.0,19.62,1.0,1.0,2.75,0.0,74,Manhattan,East Harlem North,Boro Zone
22528,2,2025-11-18,2025-11-18,N,1.0,74,166,1.0,0.86,7.9,...,1.0,11.00,1.0,1.0,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone
22529,2,2025-11-18,2025-11-18,N,1.0,74,116,1.0,2.16,15.6,...,1.0,22.23,1.0,1.0,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone


In [93]:
dq5 = dq5.groupby(['Zone'], as_index=False).agg(total_monto=("total_amount", "sum")).sort_values("total_monto", ascending=False)
dq5.head()

,Zone,total_monto
39,East Harlem North,9281.92
40,East Harlem South,6696.13
23,Central Park,2378.79
128,Washington Heights South,2139.05
85,Morningside Heights,2100.59


### Question 6

In [99]:

dq6 = dq5_tmp.merge(df_zones,left_on='PULocationID', right_on="LocationID", how='left' )
zone_pu_q6 = ['East Harlem North']
dq6 = dq6[dq6['Zone'].isin(zone_pu_q6)]
dq6 = dq6.merge(df_zones,left_on='DOLocationID', right_on="LocationID", how='left' )

dq6.columns

Index(['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime',
       'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID',
       'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge',
       'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge',
       'cbd_congestion_fee', 'LocationID_x', 'Borough_x', 'Zone_x',
       'service_zone_x', 'LocationID_y', 'Borough_y', 'Zone_y',
       'service_zone_y'],
      dtype='str')

In [100]:
dq6.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,congestion_surcharge,cbd_congestion_fee,LocationID_x,Borough_x,Zone_x,service_zone_x,LocationID_y,Borough_y,Zone_y,service_zone_y
0,2,2025-11-01,2025-11-01,N,1.0,74,42,1.0,0.74,7.2,...,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone,42,Manhattan,Central Harlem North,Boro Zone
1,2,2025-11-01,2025-11-01,N,1.0,74,42,2.0,0.95,7.2,...,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone,42,Manhattan,Central Harlem North,Boro Zone
2,2,2025-11-01,2025-11-01,N,1.0,74,262,1.0,3.40,19.8,...,2.75,0.0,74,Manhattan,East Harlem North,Boro Zone,262,Manhattan,Yorkville East,Yellow Zone
3,2,2025-11-01,2025-11-01,N,1.0,74,74,1.0,0.78,7.9,...,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone,74,Manhattan,East Harlem North,Boro Zone
4,2,2025-10-31,2025-11-01,N,1.0,74,265,1.0,16.25,61.8,...,0.00,0.0,74,Manhattan,East Harlem North,Boro Zone,265,Unknown,NaN,NaN


In [104]:
#zone_q6 = ['East Harlem North']
#dq6 = dq6[dq6['Zone'].isin(zone_q6)]

max_tip = dq6['tip_amount'].idxmax()

zone_max_tip = dq6.loc[max_tip]

print("\n Drop up Zone with max tip amount: ")
print(zone_max_tip.Zone_y)



 Drop up Zone with max tip amount: 
Yorkville West


In [9]:
! cd ..
! cd jovyan/data

/bin/bash: line 1: cd: jovyan/data: No such file or directory
